In [1]:
import os 
import findspark 
findspark.init()

# for sql
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col
from pyspark.sql.functions import sum,avg,max,count

# for time 
import time 
import datetime as dt


In [2]:
# 可以改成 *.csv 
root = '../../*.csv'
spark = SparkSession.builder.appName('eCommerce').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/28 22:25:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

ecommerce = spark.read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv(root)

In [4]:
ecommerce.createOrReplaceTempView('ecommerce_2019_oct')

In [5]:
ecommerce.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [6]:
ecommerce.show(5)

+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 01:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 01:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 01:00:01|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 01:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 01:00:01|      view|   1004775|2053013555631882655|electronics.smart...|xiaomi

### Try with smaller (day) to larger size (months) and show when the time takes to long

- Smaller days 

In [7]:
only_purchases = ecommerce.filter(col("event_type") == 'purchase')

In [8]:
only_purchases.show(5)

+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|2019-11-01 01:00:41|  purchase|  13200605|2053013557192163841|furniture.bedroom...|   NULL| 566.3|559368633|d6034fa2-41fb-4ac...|
|2019-11-01 01:01:04|  purchase|   1005161|2053013555631882655|electronics.smart...| xiaomi|211.92|513351129|e6b7ce9b-1938-4e2...|
|2019-11-01 01:04:51|  purchase|   1004856|2053013555631882655|electronics.smart...|samsung|128.42|562958505|0f039697-fedc-40f...|
|2019-11-01 01:05:34|  purchase|  26401669|2053013563651392361|                NULL|lucente|109.66|541854711|c41c44d5-ef9b-41b...|
|2019-11-01 01:06:33|  purchase|   1801881|2053013554415534427|electronics.video.tv

In [9]:
start_date = dt.datetime(2019,10,1)
end_date = dt.datetime(2019,10,2)

# filter data by date peirod 
specified_period = only_purchases.filter((col("event_time") >= start_date) & (col("event_time") < end_date))

# count number of rows
print(f'Number of rows: {specified_period.count()}')

start = time.time() 
# calculate average price for specified period
average_price_specified_period = specified_period.agg(avg("price")).collect()
print(f'average_price_specified_period: {average_price_specified_period}')
print(f'Average price for specified period ({start_date} to {end_date}): {average_price_specified_period[0][0]}')
print(f'Time taken: {time.time() - start} sec')

Number of rows: 19141


average_price_specified_period: [Row(avg(price)=324.98067760305406)]
Average price for specified period (2019-10-01 00:00:00 to 2019-10-02 00:00:00): 324.98067760305406
Time taken: 79.66258406639099 sec


- larger size (whole month)

In [10]:
# count number of rows
oct30_date = dt.datetime(2019,10,31,23,59,59)
print(f'Number of rows: {only_purchases.count()}')
all_oct_purchases = only_purchases.filter(col("event_time") <= oct30_date)
print(f'Number of rows: {all_oct_purchases.count()}') 

Number of rows: 1659788


Number of rows: 742752


In [11]:
from pyspark.sql import functions as F

# Use F.max for PySpark's max function
max_event_time = all_oct_purchases.agg(F.max("event_time")).collect()
print(f"Maximum event time: {max_event_time[0][0]}")

Maximum event time: 2019-10-31 23:59:06


In [12]:
from pyspark.sql import functions as F

# Use F.max for PySpark's max function
max_event_time = only_purchases.agg(F.max("event_time")).collect()
print(f"Maximum event time: {max_event_time[0][0]}")

Maximum event time: 2019-12-01 00:59:44


In [13]:

start = time.time() 
# calculate average price for specified period
average_proce_all = all_oct_purchases.agg(avg("price")).collect()
print(f'average_proce_all: {average_proce_all}')
print(f'Average price for whole period: {average_proce_all[0][0]}')
print(f'Time taken: {time.time() - start} sec')

average_proce_all: [Row(avg(price)=309.55203582083766)]
Average price for whole period: 309.55203582083766
Time taken: 83.80692100524902 sec


In [16]:
aggregated_data = only_purchases.groupBy("user_session") \
    .agg(
        F.max("event_time").alias("Date_order"),
        F.collect_set("user_id").alias("user_id"),  # Unique user_ids
        F.count("user_session").alias("Quantity"),
        F.sum("price").alias("money_spent")
    )

In [19]:
aggregated_data.show()

+--------------------+-------------------+-----------+--------+-----------+
|        user_session|         Date_order|    user_id|Quantity|money_spent|
+--------------------+-------------------+-----------+--------+-----------+
|000081ea-9376-4eb...|2019-10-24 11:08:58|[513622224]|       1|     131.51|
|0000c091-07d6-42b...|2019-11-18 13:48:57|[561759158]|       2|     617.26|
|0000c1d0-0429-4e0...|2019-11-07 17:58:19|[568649822]|       1|      68.97|
|000174ac-0ea3-402...|2019-10-18 12:46:20|[548449052]|       2|     499.72|
|0001b5b4-ee2c-4ce...|2019-10-05 12:06:02|[523941620]|       1|      34.88|
|0001d2b9-c9cc-474...|2019-11-05 11:08:48|[567769006]|       1|     915.85|
|0001ff74-ac24-42d...|2019-11-17 18:34:04|[532345357]|       1|     395.35|
|0002b07c-85cd-46e...|2019-10-14 12:04:26|[553453794]|       2|     366.54|
|00031054-0bd7-4c3...|2019-11-16 13:19:56|[525362834]|       1|     298.28|
|000312ec-17f5-466...|2019-11-22 10:07:52|[513531928]|       1|      90.09|
|00035f1a-f4

### Recency

In [20]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType
import datetime as dt

# Assuming 'data' is your PySpark DataFrame and 'Date_order' is a string column
study_date = dt.datetime(2019, 12, 1)

# Convert 'Date_order' to date type if it's not already
data = aggregated_data.withColumn("Date_order", F.col("Date_order").cast(DateType()))

# Calculate the difference in days
data = data.withColumn("last_purchase", F.datediff(F.lit(study_date), "Date_order"))

# Show the first few rows of the DataFrame
data.show()


+--------------------+----------+-----------+--------+-----------+-------------+
|        user_session|Date_order|    user_id|Quantity|money_spent|last_purchase|
+--------------------+----------+-----------+--------+-----------+-------------+
|000081ea-9376-4eb...|2019-10-24|[513622224]|       1|     131.51|           38|
|0000c091-07d6-42b...|2019-11-18|[561759158]|       2|     617.26|           13|
|0000c1d0-0429-4e0...|2019-11-07|[568649822]|       1|      68.97|           24|
|000174ac-0ea3-402...|2019-10-18|[548449052]|       2|     499.72|           44|
|0001b5b4-ee2c-4ce...|2019-10-05|[523941620]|       1|      34.88|           57|
|0001d2b9-c9cc-474...|2019-11-05|[567769006]|       1|     915.85|           26|
|0001ff74-ac24-42d...|2019-11-17|[532345357]|       1|     395.35|           14|
|0002b07c-85cd-46e...|2019-10-14|[553453794]|       2|     366.54|           48|
|00031054-0bd7-4c3...|2019-11-16|[525362834]|       1|     298.28|           15|
|000312ec-17f5-466...|2019-1

##### Recency – How recently did the customer purchase?
(befor days bought)
##### Frequency – How often do they purchase?
(count times)
##### Monetary Value – How much do they spend?
(per person totally amount)

In [21]:

RFM = data.groupBy("user_id") \
    .agg(
        F.min("last_purchase").alias("Recency"),
        F.count("user_id").alias("Frequency"),
        F.sum("money_spent").alias("Monetary")
    )

# Show the first few rows of the RFM DataFrame
RFM.show()


+-----------+-------+---------+------------------+
|    user_id|Recency|Frequency|          Monetary|
+-----------+-------+---------+------------------+
|[548449052]|      1|       59|          19014.07|
|[548193532]|     14|        1|           4113.36|
|[563811837]|     38|        1|            319.34|
|[542656673]|     13|       15|           4414.79|
|[520186665]|     14|        1|           2187.16|
|[512762475]|      2|        3| 542.8199999999999|
|[525277709]|      7|        1|            230.89|
|[527305704]|      6|        5| 970.6600000000001|
|[543949146]|     30|        2|           1207.21|
|[564353044]|      2|        5|            301.87|
|[570572677]|     19|        1|             29.59|
|[541788524]|      1|        5|            251.19|
|[542941877]|     13|        4|           1111.23|
|[570042604]|      9|        1|             36.04|
|[513430054]|     58|        1|            463.02|
|[514269533]|     33|        1|             138.2|
|[545325482]|     17|       66|